# PIC-SURE API tutorial using CureSC database

This is a tutorial notebook, aimed to be quickly up and running with the R PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE R API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

Databases exposed through PIC-SURE API encompass a wide heterogeneity of architectures and data organizations underneath. PIC-SURE hide this complexity and expose the different databases in the same format, allowing researchers to focus on the analysis and medical insights, thus easing the process of reproducible sciences.

### More about PIC-SURE
PIC-SURE stands for Patient-centered Information Commons: Standardized Unification of Research Elements. The API is available in two different programming languages, python and R, allowing investigators to query databases in the same way using any of those languages.

PIC-SURE is a large project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface, allowing research scientist to get quick knowledge about variables and data available for a specific data source.

The API is actively developed by the Avillach-Lab at Harvard Medical School.

GitHub repo:
* https://github.com/hms-dbmi/pic-sure-r-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-r-client



 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure you have [added your security token](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/ALS-819/Cure_Sickle_Cell#get-your-security-token). It contains explanation about how to get a security token, mandatory to access the databases.  **

# Environment set-up

### Pre-requisite
- R 3.5 or later

### Packages installation

In [ ]:
list_packages <- c("ggrepel",
                   "jsonlite", 
                   "ggplot2",
                   "plyr",
                   "dplyr",
                   "tidyr",
                   "purrr",
                   "urltools",
                   "devtools")

for (package in list_packages){
     if(! package %in% installed.packages()){
         install.packages(package, dependencies = TRUE)
     }
     library(package, character.only = TRUE)
}

#### Installing latest R PIC-SURE API libraries from github

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")

devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)

##### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE network

You will need the following information before connecting to the pic-sure network:
* resource_id = Id of the resource that you are trying to access. You can leave the default value for this project.
* token_file = A flat file called token.txt should contain the token retrieved from you user profile in picsure-ui.  The needs to be located at the R root folder.

In [ ]:
token_file <- "token.txt"
my_token <- scan(token_file, what = "character")

In [ ]:
connection <- picsure::connect_local(token = my_token)

In [ ]:
resource <- hpds::get.resource(connection,
                               resourceUUID = picsure::list.resources(connection))

In [ ]:
picsure::list.resources(connection)

Two objects are created here: a `connection` and a `resource` object, using respectively the `picsure` and `hpds` libraries. 

As we will only be using one single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter** (FYI, the `connection` object is useful to get access to different databases stored in different resources). 

It is connected to the specific data source ID we specified, and enables to query and retrieve data from this source.

## Getting help with the R PIC-SURE API

The `?` operator prints out helper message for any PIC-SURE library function.

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to get a quick idea of which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance offers the possibility to retrieve matching records according to a specific term, or to retrieve information about all available variables, using the `find.in.dictionary()` function. For instance, looking for variables containing the term 'Smoke' is done this way: 

In [ ]:
dictionary_search <- hpds::find.in.dictionary(resource, "Avascular necrosis")

Subsequently, objects created by the `find.in.dictionary` exposes the search result using 4 different methods: `extract.count()`, `extract.keys()`, `extract.entries()`, and `extract.DataFrame()`. 

In [ ]:
print(list("Count"   = hpds::extract.count(dictionary_search), 
           "Keys"    = hpds::extract.keys(dictionary_search)[1:2],
           "Entries" = hpds::extract.entries(dictionary_search)[1:2]))

In [ ]:
hpds::extract.dataframe(dictionary_search) %>% tail()

**`hpds::extract.dataframe()` enables to get the result of the dictionary search in a data.frame format.**

The dictionary provide various information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if categorical, False if continuous/numerical
- min/max: only provided for non-categorical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

Hence, it enables to:
* Use the various variables information as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variables names, to be used in the query, as shown below.


Variable names, as currently implemented in the API, aren't straightforward to use.
1. Very long
2. Presence of backslashes that requires modification right after copy-pasting. 

However, using the dictionary to select variables can help to deal with this. Let's say we want to retrieve every variable from the different substudies available in the resource (ie Cure Sickle Cell related studies). Hence, one way to proceed would be to retrieve the whole dictionary for those variables in the form of a data.frame, as below:

In [ ]:
plain_variablesDict <- hpds::find.in.dictionary(resource, "CIBMTR - Cure Sickle Cell Disease") %>% hpds::extract.dataframe()

Moreover, using the `hpds::find.in.dictionary` function without arguments return every entries, as shown in the help documentation. *As for now, this takes a long time in the R PIC-SURE API implementation, and it will probably be fixed in the latter version of the API*

In [ ]:
plain_variablesDict[10:20,]

The dictionary currently returned by the API provide various information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if categorical, False if continuous/numerical
- min/max: only provided for non-categorical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

### Extract Full Data Dictionary to CSV

Using the `hpds::find.in.dictionary` we can extact the entire data dictionary by performing an empty search:
*This is a long running process.

In [ ]:
fullVariableDict <- hpds::find.in.dictionary(resource, "") %>% hpds::extract.dataframe()

Check that the fullVariableDict dataframe contains some values.

In [ ]:
fullVariableDict[0:5,]

We can than write the data frame that contains the full data dictionary to a csv file

In [ ]:
dataDictFile <- "data_dictionary.csv"
write.csv(fullVariableDict, dataDictFile, row.names = FALSE)

You should now see a data_dictionary.csv in the file explorer.

### Parsing variable names

Though helpful, we can use a simple function, `get_multiIndex`, defined in `R_lib/utils.R` to add a little more information and ease working with variables names. 

Although not an official feature of the API, such functionality illustrate how to quickly scan and select groups of related variables.

Printing part of the "parsed names" Dictionary allows to quickly see the tree-like organisation of the variables. Moreover, original and simplified variable names are now stored respectively in the "varName" and "simplified_varName" columns (simplified variable names is simply the last component of the variable name, which usually makes the most sense to know what each variable is about).

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict <- get_multiIndex_variablesDict(plain_variablesDict)
head(variablesDict)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in every variables pertaining to the "Avascular necrosis" of the "CIBMTR" study.

In [ ]:
mask_necrosis <- grepl("Avascular necrosis", variablesDict[["simplified_name"]])
mask_necrosis

In [ ]:
mask_study <- variablesDict[,1] == "CIBMTR - Cure Sickle Cell Disease"
mask_necrosis <- grepl("Avascular necrosis", variablesDict[["simplified_name"]])
more_variables <- variablesDict[mask_study & mask_necrosis,]
more_variables

Although pretty simple, it can be easily combined with other filters to quickly select necessary variables.

## Querying and retrieving data

Beside from the dictionary, the second cornerstone of the API are the `query` functions (`hpds::query.anyof`, `hpds::query.select`, `hpds::query.filter`, `hpds::query.require`). They are the entering point to retrieve data from the resource.

First, we need to create a query object.

In [ ]:
my_query <- hpds::new.query(resource = resource)

The query object created will be then be passed to the different query functions to build the query: `hpds::query.anyof`, `hpds::query.select`, `hpds::query.filter`, `hpds::query.require`. Each of those methods accept a query object, a list of variable names, and eventual additional parameters

- The `query.select.add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, without any record (ie subjects/rows) subsetting.

- The `query.require.add()` method accept variable names as string or list of strings as argument, and will allow the query to return all the variables passed, and only records that do not contain any null values for those variables.

- The `query.anyof.add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, and only records that do contain at least one non-null value for those variables.

- The `query.filter.add()` method accept variable names a variable name as strings as argument, plus additional values to filter on that given variable. The query will return this variable and only the records that do match this filter.

All those 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

### Building the query

Let's say we want to select a cohort from the "CIBMTR study" for males with Avascular necrosis

In [ ]:
# Selecting all variables from "CIBMTR" study
mask_study = variablesDict[["level_0"]] == "CIBMTR - Cure Sickle Cell Disease"
varnames = variablesDict[mask_study, ]$name %>% as.list()

In [ ]:
sex_var <- variablesDict[variablesDict["simplified_name"] == "Sex", ]$name 

avascular_necrosis_varname <- variablesDict[variablesDict["simplified_name"] == "Avascular necrosis", ]$name 
values <- variablesDict[mask_study, "categoryValues"]

In [ ]:
my_query <- hpds::new.query(resource = resource)
hpds::query.select.add(my_query, keys = sex_var)
hpds::query.filter.add(my_query, sex_var, "Male")

hpds::query.select.add(my_query, keys = avascular_necrosis_varname)
hpds::query.filter.add(my_query, avascular_necrosis_varname, "Yes")


## Retrieving the data

Once our query object is finally built, we use the `query.run` function to retrieve the data corresponding to our query

In [ ]:
my_df <- hpds::query.run(my_query, result.type = "dataframe")

In [ ]:
dim(my_df)

In [ ]:
head(my_df)